In [504]:
import sympy as sy
import numpy as np
import math
import random

from enum import Enum
from typing import List


In [505]:
class Signs(Enum):
    LESS = 1
    EQUAL = 2
    GREAT = 3


class TaskType(Enum):
    MIN = 1
    MAX = 2


class Inequality:
    def __init__(self, coeff: List[int], sign: Signs, free: int):
        self.coeff = coeff
        self.sign = sign
        self.free = free


class Task:
    def __init__(self, main_coeff: List[int], task_type: TaskType, inequals: List[Inequality]):
        self.main_coeff = main_coeff
        self.task_type = task_type
        self.inequals = inequals


In [506]:
def check(actual, expected):
    if isinstance(actual, list):
        ok = True
        for i in range(len(actual)):
            if actual[i] != expected[i]:
                ok = False
        if not ok:
            print('points are different')
    else:
        if actual != expected:
            print(f'not equal. expected: {expected}, got: {actual}')



In [507]:
# print(random.randint(0, 2))


In [508]:
def to_rational_array(a):
    sa = a.shape
    return np.array([sy.Rational(x) for x in a.reshape(-1)]).reshape(sa)


In [509]:
def simplex1(matrix_a, column_b, functional, initial_basis):
    functional = np.append(functional, 0)

    matrix_a = to_rational_array(matrix_a)
    column_b = to_rational_array(column_b)
    functional = to_rational_array(functional)

    data = np.c_[matrix_a, column_b * -1]

    n = matrix_a.shape[1]
    m = matrix_a.shape[0]

    print(data)
    print(functional)
    print(data.shape)

    # TODO rewrite gauss
    # TODO преобразование initial_basis

    # initial_basis2 = [0] * n
    # for t in initial_basis:
    #     initial_basis2[t] = 1
    # initial_basis = initial_basis2

    print(initial_basis)


    # to ones on basis
    m_to_basis = [-1] * m
    # cnt = 0
    # for ind in range(n):
    #     if initial_basis[ind] == 1:
    #         if data[cnt][ind] == 0:
    #             for i in range(cnt + 1, m):
    #                 if data[i][ind] != 0:
    #                     data[cnt] += data[i]
    #                     break
    #         data[cnt] /= data[cnt][ind]
    #
    #         for i in range(m):
    #             if i == cnt:
    #                 continue
    #             data[i] -= data[cnt] * data[i][ind]
    #         m_to_basis[cnt] = ind
    #         cnt += 1


    for j in range(m):
        cnt = j
        ind = initial_basis[j]
        if data[cnt][ind] == 0:
            for i in range(cnt + 1, m):
                if data[i][ind] != 0:
                    data[cnt] += data[i]
                    break
        data[cnt] /= data[cnt][ind]

        for i in range(m):
            if i == cnt:
                continue
            data[i] -= data[cnt] * data[i][ind]
        m_to_basis[cnt] = ind


    for i in range(m):
        j = m_to_basis[i]
        functional -= data[i] * (functional[j] / data[i][j])

    print("after gauss")
    print(data)
    print(functional)
    print("----------")

    for cyc in range(100):
        ind_in_basis, ind_out_basis = -1, -1
        # select random pair
        # ind_in = list(filter(lambda x: functional[x] > 0, range(n)))
        for i in range(n):
            if functional[i] > 0:
                ind_in_basis = i
                break
        if ind_in_basis == -1:
        # if len(ind_in) == 0:
            print('complete')
            break
        # ind_in_basis = ind_in[random.randint(0, len(ind_in) - 1)]
        prev_rez = -math.inf
        for j in range(m):
            if data[j][ind_in_basis] > 0:
                delta = data[j][-1] / data[j][ind_in_basis]
                # if 0 >= delta > prev_rez:
                if 0 >= delta > prev_rez:
                # if 0 >= delta > prev_rez or \
                        # (delta == prev_rez and (m_to_basis[ind_out_basis] > m_to_basis[j] or ind_out_basis == -1)):
                # if delta < prev_rez:
                    ind_out_basis = j
                    prev_rez = delta
        # if prev_rez == 0:
        #     aa = []
        #     for j in range(m):
        #         if data[j][ind_in_basis] != 0:
        #             delta = data[j][-1] / data[j][ind_in_basis]
        #             if delta <= 0:
        #                 aa.append(j)
        #     ind_out_basis = aa[random.randint(0, len(aa) - 1)]
        #     print("heh: ", ind_out_basis)
        if ind_out_basis == -1:
            print('unlimited')
            print(ind_in_basis)
            break

        m_to_basis[ind_out_basis] = ind_in_basis

        data[ind_out_basis] /= data[ind_out_basis][ind_in_basis]
        for i in range(m):
            if i != ind_out_basis:
                data[i] -= data[ind_out_basis] * data[i][ind_in_basis]

        functional -= data[ind_out_basis] * functional[ind_in_basis]

        print(cyc)
        print(prev_rez)
        print(ind_in_basis, ind_out_basis)
        print(data)
        print(functional)
        print("----------")

    opt_point = [0] * n
    for i in range(m):
        opt_point[m_to_basis[i]] = -data[i][-1]

    mbs = sorted(m_to_basis)
    print("m:, ", m_to_basis, mbs)

    return -functional[-1], opt_point, mbs


In [510]:
def solve(task: Task, initial_basis: List[int] = None):
    # transform data from dsl to np.array
    matrix_a = np.array(
        [inequality.coeff for inequality in task.inequals],
        dtype=np.float64
    )
    column_b = np.array(
        [inequality.free for inequality in task.inequals],
        dtype=np.float64
    )
    main_functional = np.array(task.main_coeff, dtype=np.float64)
    if task.task_type == TaskType.MIN:
        main_functional *= -1

    m = matrix_a.shape[0]
    n = matrix_a.shape[1]

    for i in range(m):
        inequality = task.inequals[i]
        if inequality.sign == Signs.GREAT:
            matrix_a[i] *= -1
            column_b[i] *= -1
            # TODO check: sign changed from > to <
        if inequality.sign != Signs.EQUAL:
            addition_column = np.zeros(m)
            addition_column[i] = 1
            matrix_a = np.c_[
                matrix_a,
                addition_column
            ]
            main_functional = np.append(main_functional, 0)

    n2 = matrix_a.shape[1]

    # выполняем метод искуственного базиса, если начальный базис не передан явно
    if initial_basis is None:
        matrix_y = np.eye(m)
        for i in range(m):
            if column_b[i] < 0:
                matrix_y[i][i] = -1
        a_matrix_ay = np.c_[matrix_a, matrix_y]
        a_functional = np.array([0] * n2 + [-1] * m)
        # a_in_func = a_functional * -1
        a_in_func = list(range(n2, n2 + m))
        print("ib0:", a_in_func)
        opt_value, point, mbs = simplex1(a_matrix_ay, column_b, a_functional, a_in_func)

        print(">", opt_value, point)
        if abs(opt_value) != 0:
            print("not feasible")
            return 0, []

        # выбирается ровно m базисных переменных
        # сначала в которых значение не 0, затем добираются нулевые
        # initial_basis = [
        #                     ind for _val, ind in
        #                     filter(
        #                         lambda val: val[0] == 0,
        #                         zip(point[:-m], range(n2))
        #                     )
        #                 ][:(n2 - m)]
        # initial_basis = list(set(range(n2)) - set(initial_basis))
        initial_basis = mbs
        print('ib: ', initial_basis)

    opt_value, opt_point, _mbs2 = simplex1(matrix_a, column_b, main_functional, initial_basis)

    opt_value *= 1 if task.task_type == TaskType.MIN else -1
    opt_point = opt_point[:n]

    print("optimal value", opt_value)
    print("in point:", opt_point)

    return opt_value, opt_point


In [511]:
task_1 = Task(
    [-6, -1, -4, 5], TaskType.MIN,
    [
        Inequality([3, 1, -1, 1], Signs.EQUAL, 4),
        Inequality([5, 1, 1, -1], Signs.EQUAL, 4)
    ]
)

result_1_value, result_1_point = solve(
    task_1,
    [0, 3]
)

check(result_1_value, -4)
check(result_1_point, [0, 4, 0, 0])

[[3 1 -1 1 -4]
 [5 1 1 -1 -4]]
[6 1 4 -5 0]
(2, 5)
[0, 3]
after gauss
[[1 1/4 0 0 -1]
 [0 1/4 -1 1 -1]]
[0 3/4 -1 0 1]
----------
0
-4
1 0
[[4 1 0 0 -4]
 [-1 0 -1 1 0]]
[-3 0 -1 0 4]
----------
complete
m:,  [1, 3] [1, 3]
optimal value -4
in point: [0, 4, 0, 0]


In [512]:
result_1_2_value, result_1_2_point = solve(task_1)

check(result_1_2_value, -4)
check(result_1_2_point, [0, 4, 0, 0])


ib0: [4, 5]
[[3 1 -1 1 1 0 -4]
 [5 1 1 -1 0 1 -4]]
[0 0 0 0 -1 -1 0]
(2, 7)
[4, 5]
after gauss
[[3 1 -1 1 1 0 -4]
 [5 1 1 -1 0 1 -4]]
[8 2 0 0 0 0 -8]
----------
0
-4/5
0 1
[[0 2/5 -8/5 8/5 1 -3/5 -8/5]
 [1 1/5 1/5 -1/5 0 1/5 -4/5]]
[0 2/5 -8/5 8/5 0 -8/5 -8/5]
----------
1
-4
1 0
[[0 1 -4 4 5/2 -3/2 -4]
 [1 0 1 -1 -1/2 1/2 0]]
[0 0 0 0 -1 -1 0]
----------
complete
m:,  [1, 0] [0, 1]
> 0 [0, 4, 0, 0, 0, 0]
ib:  [0, 1]
[[3 1 -1 1 -4]
 [5 1 1 -1 -4]]
[6 1 4 -5 0]
(2, 5)
[0, 1]
after gauss
[[1 0 1 -1 0]
 [0 1 -4 4 -4]]
[0 0 2 -3 4]
----------
0
0
2 0
[[1 0 1 -1 0]
 [4 1 0 0 -4]]
[-2 0 0 -1 4]
----------
complete
m:,  [2, 1] [1, 2]
optimal value -4
in point: [0, 4, 0, 0]


In [513]:
#TODO дробные числа

task_2 = Task(
    [-1, -2, -3, 1], TaskType.MIN,
    [
        Inequality([1, -3, -1, -2], Signs.EQUAL, -4),
        Inequality([1, -1, 1, 0], Signs.EQUAL, 0)
    ]
)

result_2_value, result_2_point = solve(
    task_2,
    [1, 2]
)

check(result_2_value, -6)
check(result_2_point, [2, 2, 0, 0])


[[1 -3 -1 -2 4]
 [1 -1 1 0 0]]
[1 2 3 -1 0]
(2, 5)
[1, 2]
after gauss
[[-1/2 1 0 1/2 -1]
 [1/2 0 1 1/2 -1]]
[1/2 0 0 -7/2 5]
----------
0
-2
0 1
[[0 1 1 1 -2]
 [1 0 2 1 -2]]
[0 0 -1 -4 6]
----------
complete
m:,  [1, 0] [0, 1]
optimal value -6
in point: [2, 2, 0, 0]


In [514]:
task_2_max = Task(
    [-1, -2, -3, 1], TaskType.MAX,
    [
        Inequality([1, -3, -1, -2], Signs.EQUAL, -4),
        Inequality([1, -1, 1, 0], Signs.EQUAL, 0)
    ]
)

result_2_max_value, result_2_max_point = solve(task_2_max)

check(result_2_max_value, 2)
check(result_2_max_point, [0, 0, 0, 2])


ib0: [4, 5]
[[1 -3 -1 -2 -1 0 4]
 [1 -1 1 0 0 1 0]]
[0 0 0 0 -1 -1 0]
(2, 7)
[4, 5]
after gauss
[[-1 3 1 2 1 0 -4]
 [1 -1 1 0 0 1 0]]
[0 2 2 2 0 0 -4]
----------
0
-4/3
1 0
[[-1/3 1 1/3 2/3 1/3 0 -4/3]
 [2/3 0 4/3 2/3 1/3 1 -4/3]]
[2/3 0 4/3 2/3 -2/3 0 -4/3]
----------
1
-2
0 1
[[0 1 1 1 1/2 1/2 -2]
 [1 0 2 1 1/2 3/2 -2]]
[0 0 0 0 -1 -1 0]
----------
complete
m:,  [1, 0] [0, 1]
> 0 [2, 2, 0, 0, 0, 0]
ib:  [0, 1]
[[1 -3 -1 -2 4]
 [1 -1 1 0 0]]
[-1 -2 -3 1 0]
(2, 5)
[0, 1]
after gauss
[[1 0 2 1 -2]
 [0 1 1 1 -2]]
[0 0 1 4 -6]
----------
0
-1
2 0
[[1/2 0 1 1/2 -1]
 [-1/2 1 0 1/2 -1]]
[-1/2 0 0 7/2 -5]
----------
1
-2
3 0
[[1 0 2 1 -2]
 [-1 1 -1 0 0]]
[-4 0 -7 0 2]
----------
complete
m:,  [3, 1] [1, 3]
optimal value 2
in point: [0, 0, 0, 2]


In [515]:
task_3 = Task(
    [-1, -2, -1, 3, -1], TaskType.MIN,
    [
        Inequality([1, 1, 0, 2, 1], Signs.EQUAL, 5),
        Inequality([1, 1, 1, 3, 2], Signs.EQUAL, 9),
        Inequality([0, 1, 1, 2, 1], Signs.EQUAL, 6)
    ]
)

result_3_value, result_3_point = solve(task_3)

check(result_3_value, -11)
check(result_3_point, [3, 2, 4, 0, 0])


ib0: [5, 6, 7]
[[1 1 0 2 1 1 0 0 -5]
 [1 1 1 3 2 0 1 0 -9]
 [0 1 1 2 1 0 0 1 -6]]
[0 0 0 0 0 -1 -1 -1 0]
(3, 9)
[5, 6, 7]
after gauss
[[1 1 0 2 1 1 0 0 -5]
 [1 1 1 3 2 0 1 0 -9]
 [0 1 1 2 1 0 0 1 -6]]
[2 3 2 7 4 0 0 0 -20]
----------
0
-5
0 0
[[1 1 0 2 1 1 0 0 -5]
 [0 0 1 1 1 -1 1 0 -4]
 [0 1 1 2 1 0 0 1 -6]]
[0 1 2 3 2 -2 0 0 -10]
----------
1
-5
1 0
[[1 1 0 2 1 1 0 0 -5]
 [0 0 1 1 1 -1 1 0 -4]
 [-1 0 1 0 0 -1 0 1 -1]]
[-1 0 2 1 1 -3 0 0 -5]
----------
2
-1
2 2
[[1 1 0 2 1 1 0 0 -5]
 [1 0 0 1 1 0 1 -1 -3]
 [-1 0 1 0 0 -1 0 1 -1]]
[1 0 0 1 1 -1 0 -2 -3]
----------
3
-3
0 1
[[0 1 0 1 0 1 -1 1 -2]
 [1 0 0 1 1 0 1 -1 -3]
 [0 0 1 1 1 -1 1 0 -4]]
[0 0 0 0 0 -1 -1 -1 0]
----------
complete
m:,  [1, 0, 2] [0, 1, 2]
> 0 [3, 2, 4, 0, 0, 0, 0, 0]
ib:  [0, 1, 2]
[[1 1 0 2 1 -5]
 [1 1 1 3 2 -9]
 [0 1 1 2 1 -6]]
[1 2 1 -3 1 0]
(3, 6)
[0, 1, 2]
after gauss
[[1 0 0 1 1 -3]
 [0 1 0 1 0 -2]
 [0 0 1 1 1 -4]]
[0 0 0 -7 -1 11]
----------
complete
m:,  [0, 1, 2] [0, 1, 2]
optimal value -11
in point: [3, 2,

In [516]:
task4 = Task(
    [-1, -1, -1, 1, -1], TaskType.MIN,
    [
        Inequality([1, 1, 2, 0, 0], Signs.EQUAL, 4),
        Inequality([0, -2, -2, 1, -1], Signs.EQUAL, -6),
        Inequality([1, -1, 6, 1, 1], Signs.EQUAL, 12)
    ]
)

result_4_value, result_4_point = solve(task4)

check(result_4_value, -10)
check(result_4_point, [4, 0, 0, 6, 12])


ib0: [5, 6, 7]
[[1 1 2 0 0 1 0 0 -4]
 [0 -2 -2 1 -1 0 -1 0 6]
 [1 -1 6 1 1 0 0 1 -12]]
[0 0 0 0 0 -1 -1 -1 0]
(3, 9)
[5, 6, 7]
after gauss
[[1 1 2 0 0 1 0 0 -4]
 [0 2 2 -1 1 0 1 0 -6]
 [1 -1 6 1 1 0 0 1 -12]]
[2 2 10 0 2 0 0 0 -22]
----------
0
-4
0 0
[[1 1 2 0 0 1 0 0 -4]
 [0 2 2 -1 1 0 1 0 -6]
 [0 -2 4 1 1 -1 0 1 -8]]
[0 0 6 0 2 -2 0 0 -14]
----------
1
-2
2 0
[[1/2 1/2 1 0 0 1/2 0 0 -2]
 [-1 1 0 -1 1 -1 1 0 -2]
 [-2 -4 0 1 1 -3 0 1 0]]
[-3 -3 0 0 2 -5 0 0 -2]
----------
2
0
4 2
[[1/2 1/2 1 0 0 1/2 0 0 -2]
 [1 5 0 -2 0 2 1 -1 -2]
 [-2 -4 0 1 1 -3 0 1 0]]
[1 5 0 -2 0 1 0 -2 -2]
----------
3
-2
0 1
[[0 -2 1 1 0 -1/2 -1/2 1/2 -1]
 [1 5 0 -2 0 2 1 -1 -2]
 [0 6 0 -3 1 1 2 -1 -4]]
[0 0 0 0 0 -1 -1 -1 0]
----------
complete
m:,  [2, 0, 4] [0, 2, 4]
> 0 [2, 0, 1, 0, 4, 0, 0, 0]
ib:  [0, 2, 4]
[[1 1 2 0 0 -4]
 [0 -2 -2 1 -1 6]
 [1 -1 6 1 1 -12]]
[1 1 1 -1 1 0]
(3, 6)
[0, 2, 4]
after gauss
[[1 5 0 -2 0 -2]
 [0 -2 1 1 0 -1]
 [0 6 0 -3 1 -4]]
[0 -8 0 3 0 7]
----------
0
-1
3 1
[[1 1 2 0 0 -4]
 [

In [517]:
#TODO побороть зацикливание

task5 = Task(
    [-1, 4, -3, 10], TaskType.MIN,
    [
        Inequality([1, 1, -1, -10], Signs.EQUAL, 0),
        Inequality([1, 14, 10, -10], Signs.EQUAL, 11),
    ]
)

result_5_value, result_5_point = solve(
    task5,
    # [1, 3]
)

check(result_5_value, -4)
check(result_5_point, [1, 0, 1, 0])


ib0: [4, 5]
[[1 1 -1 -10 1 0 0]
 [1 14 10 -10 0 1 -11]]
[0 0 0 0 -1 -1 0]
(2, 7)
[4, 5]
after gauss
[[1 1 -1 -10 1 0 0]
 [1 14 10 -10 0 1 -11]]
[2 15 9 -20 0 0 -11]
----------
0
0
0 0
[[1 1 -1 -10 1 0 0]
 [0 13 11 0 -1 1 -11]]
[0 13 11 0 -2 0 -11]
----------
1
0
1 0
[[1 1 -1 -10 1 0 0]
 [-13 0 24 130 -14 1 -11]]
[-13 0 24 130 -15 0 -11]
----------
2
-11/24
2 1
[[11/24 1 0 -55/12 5/12 1/24 -11/24]
 [-13/24 0 1 65/12 -7/12 1/24 -11/24]]
[0 0 0 0 -1 -1 0]
----------
complete
m:,  [1, 2] [1, 2]
> 0 [0, 11/24, 11/24, 0, 0, 0]
ib:  [1, 2]
[[1 1 -1 -10 0]
 [1 14 10 -10 -11]]
[1 -4 3 -10 0]
(2, 5)
[1, 2]
after gauss
[[11/24 1 0 -55/12 -11/24]
 [-13/24 0 1 65/12 -11/24]]
[107/24 0 0 -535/12 -11/24]
----------
0
-1
0 0
[[1 24/11 0 -10 -1]
 [0 13/11 1 0 -1]]
[0 -107/11 0 0 4]
----------
complete
m:,  [0, 2] [0, 2]
optimal value -4
in point: [1, 0, 1, 0]


In [518]:
task_6 = Task(
    [-1, 5, 1, -1], TaskType.MIN,
    [
        Inequality([1, 3, 3, 1], Signs.LESS, 3),
        Inequality([0, 2, 3, -1], Signs.LESS, 4),
    ]
)

result_6_value, result_6_point = solve(task_6)

check(result_6_value, -3)
check(result_6_point, [3, 0, 0, 0])


ib0: [6, 7]
[[1 3 3 1 1 0 1 0 -3]
 [0 2 3 -1 0 1 0 1 -4]]
[0 0 0 0 0 0 -1 -1 0]
(2, 9)
[6, 7]
after gauss
[[1 3 3 1 1 0 1 0 -3]
 [0 2 3 -1 0 1 0 1 -4]]
[1 5 6 0 1 1 0 0 -7]
----------
0
-3
0 0
[[1 3 3 1 1 0 1 0 -3]
 [0 2 3 -1 0 1 0 1 -4]]
[0 2 3 -1 0 1 -1 0 -4]
----------
1
-1
1 0
[[1/3 1 1 1/3 1/3 0 1/3 0 -1]
 [-2/3 0 1 -5/3 -2/3 1 -2/3 1 -2]]
[-2/3 0 1 -5/3 -2/3 1 -5/3 0 -2]
----------
2
-1
2 0
[[1/3 1 1 1/3 1/3 0 1/3 0 -1]
 [-1 -1 0 -2 -1 1 -1 1 -1]]
[-1 -1 0 -2 -1 1 -2 0 -1]
----------
3
-1
5 1
[[1/3 1 1 1/3 1/3 0 1/3 0 -1]
 [-1 -1 0 -2 -1 1 -1 1 -1]]
[0 0 0 0 0 0 -1 -1 0]
----------
complete
m:,  [2, 5] [2, 5]
> 0 [0, 0, 1, 0, 0, 1, 0, 0]
ib:  [2, 5]
[[1 3 3 1 1 0 -3]
 [0 2 3 -1 0 1 -4]]
[1 -5 -1 1 0 0 0]
(2, 7)
[2, 5]
after gauss
[[1/3 1 1 1/3 1/3 0 -1]
 [-1 -1 0 -2 -1 1 -1]]
[4/3 -4 0 4/3 1/3 0 -1]
----------
0
-3
0 0
[[1 3 3 1 1 0 -3]
 [0 2 3 -1 0 1 -4]]
[0 -8 -4 0 -1 0 3]
----------
complete
m:,  [0, 5] [0, 5]
optimal value -3
in point: [3, 0, 0, 0]


In [519]:
task_7 = Task(
    [-1, -1, 1, -1, 2], TaskType.MIN,
    [
        Inequality([3, 1, 1, 1, -2], Signs.EQUAL, 10),
        Inequality([6, 1, 2, 3, -4], Signs.EQUAL, 20),
        Inequality([10, 1, 3, 6, -7], Signs.EQUAL, 30)
    ]
)

result_7_value, result_7_point = solve(task_7)

check(result_7_value, 10)
check(result_7_point, [0, 0, 10, 0, 0])


ib0: [5, 6, 7]
[[3 1 1 1 -2 1 0 0 -10]
 [6 1 2 3 -4 0 1 0 -20]
 [10 1 3 6 -7 0 0 1 -30]]
[0 0 0 0 0 -1 -1 -1 0]
(3, 9)
[5, 6, 7]
after gauss
[[3 1 1 1 -2 1 0 0 -10]
 [6 1 2 3 -4 0 1 0 -20]
 [10 1 3 6 -7 0 0 1 -30]]
[19 3 6 10 -13 0 0 0 -60]
----------
0
-3
0 2
[[0 7/10 1/10 -4/5 1/10 1 0 -3/10 -1]
 [0 2/5 1/5 -3/5 1/5 0 1 -3/5 -2]
 [1 1/10 3/10 3/5 -7/10 0 0 1/10 -3]]
[0 11/10 3/10 -7/5 3/10 0 0 -19/10 -3]
----------
1
-10/7
1 0
[[0 1 1/7 -8/7 1/7 10/7 0 -3/7 -10/7]
 [0 0 1/7 -1/7 1/7 -4/7 1 -3/7 -10/7]
 [1 0 2/7 5/7 -5/7 -1/7 0 1/7 -20/7]]
[0 0 1/7 -1/7 1/7 -11/7 0 -10/7 -10/7]
----------
2
-10
2 0
[[0 7 1 -8 1 10 0 -3 -10]
 [0 -1 0 1 0 -2 1 0 0]
 [1 -2 0 3 -1 -3 0 1 0]]
[0 -1 0 1 0 -3 0 -1 0]
----------
3
0
3 1
[[0 -1 1 0 1 -6 8 -3 -10]
 [0 -1 0 1 0 -2 1 0 0]
 [1 1 0 0 -1 3 -3 1 0]]
[0 0 0 0 0 -1 -1 -1 0]
----------
complete
m:,  [2, 3, 0] [0, 2, 3]
> 0 [0, 0, 10, 0, 0, 0, 0, 0]
ib:  [0, 2, 3]
[[3 1 1 1 -2 -10]
 [6 1 2 3 -4 -20]
 [10 1 3 6 -7 -30]]
[1 1 -1 1 -2 0]
(3, 6)
[0, 2, 3]
af